# Load datasets

In [1]:
from datasets import load_dataset, load_metric

#eduskunta = "data/eduskunnanpuheet/train.csv"
singlespeaker = "data/singlespeaker/train.csv"
commonvoice = "data/commonvoice/train.csv"
speechcollector = "data/speechcollector/train.csv"
voxpopuli = "data/fi/train.csv"

test = "data/commonvoice/test.csv"

dataset = load_dataset('csv', data_files = {'train' : [singlespeaker, commonvoice, speechcollector, voxpopuli],
                               'test' : test
                              })

Using custom data configuration default-cf72a1060303bc9b
Reusing dataset csv (/home/sampo/.cache/huggingface/datasets/csv/default-cf72a1060303bc9b/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [2]:
common_voice_train = dataset['train']
common_voice_test = dataset['test']

In [3]:
import random
import pandas as pd
from IPython.display import display, HTML
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\...\…\–\é]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

from datasets import ClassLabel

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(common_voice_train.remove_columns(["path"]))

common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)




,sentence
0,tänä päivänä erityisesti haluan nostaa esiin nykyiset ruoan tuotanto ja kulutustapamme jotka ovat täysin kestämättömällä pohjalla.
1,olemme koko ajan korostaneet sitä että investoinneille täytyy raivata tilaa talouteen täytyy saada happea työllisyys täytyy saada paranemaan ja niin edelleen.
2,nuo sievät linnut olivat heränneet aikaisemmin kuin me väsyneet ihmiset aamusiivouksia tehdessään pudotti pääskyemo jotakin korvaani
3,työssä komissaari vain panee täytäntöön euroopan unionin perussopimuksia ja direktiivejä joissa puhutaan siitä että miehet ja naiset ovat tasavertaisia.
4,arvoisa puhemies arvoisa komission jäsen en ole enää pysynyt laskuissa siitä kuinka monetta kertaa pelkästään tänä vuonna käsittelemme valko venäjän lukaenkan brutaalin hallinnon toimia maan omia kansalaisia vastaan.
5,mitä mielenterveys on
6,päivitetyllä direktiivillä yhdenmukaistetaan juomaveden laatuvaatimuksia niin että vaatimukset perustuvat uusimpaan tieteelliseen tietoon maailman terveysjärjestön suositusten mukaisesti.
7,olen nyt rauhoittunut luulen itseni tässä hetkessä voivan kuollakin
8,tämä osoittaa yksinkertaisella selkeällä tavalla mitä tarkoittaa kun eu tekee enemmän yhdessä.
9,on kuitenkin tärkeää muistaa että työvoimapalvelut ja työttömyysturva on järjestetty eri tavoin eri maissa ja ne myös kuuluvat kansalliseen päätäntävaltaan. siksi mielestäni ainoa mahdollisuus edetä tässä on vapaaehtoisuuden pohjalta mutta parhaita käytäntöjä vaihtamalla päästään varmastikin jo ihan hyvin eteenpäin.


In [4]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

In [5]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)

{'f': 0, 's': 1, 'd': 2, 'q': 3, 't': 4, 'e': 5, 'r': 6, 'z': 7, 'p': 8, 'x': 9, 'm': 10, 'j': 11, 'g': 12, 'l': 13, 'a': 14, 'o': 15, 'y': 16, 'k': 17, 'ö': 18, 'n': 19, 'v': 20, 'c': 21, 'w': 22, 'ä': 23, 'u': 24, 'h': 25, 'b': 26, ' ': 27, 'i': 28, 'å': 29}


In [6]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [7]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

32


In [8]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [9]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [10]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [11]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [12]:
import torchaudio
import librosa
batch = common_voice_train[6666]
print(librosa.load(batch['path'])[0].shape)

/home/sampo/anaconda3/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


(139264,)


In [13]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

KeyboardInterrupt: 

In [ ]:
import librosa
import numpy as np

def resample(batch):
    sr = batch["sampling_rate"]
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), sr, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

common_voice_train = common_voice_train.map(resample, num_proc=1)
common_voice_test = common_voice_test.map(resample, num_proc=1)

In [15]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train))

ipd.Audio(data=np.asarray(common_voice_train[rand_int]["speech"]), rate=16000)

In [16]:
rand_int = random.randint(0, len(common_voice_train))

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

Target text: arvonlisäveroista muodostuu helposti lisäverotus työlle silloin kun lopputuote ei sisällä juurikaan mahdollisuutta vähentää materiaalikustannuksia 
Input array shape: (141120,)
Sampling rate: 16000


In [17]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

/home/sampo/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [18]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [19]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [20]:
wer_metric = load_metric("wer")

In [21]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [22]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-100k-voxpopuli",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-100k-voxpopuli and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
model.freeze_feature_extractor()

In [24]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./run_voxpopuli",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=4,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=2e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [25]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [26]:
trainer.train()

/home/sampo/anaconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
400,4.696900,2.915485,1.000000,37.922000,11.286000
800,1.403700,0.343172,0.539137,37.648800,11.368000


RuntimeError: [enforce fail at inline_container.cc:274] . unexpected pos 4342208 vs 4342096